* re.sub(pattern, replace, str) replace pattern in str with replace
* \1 stands for group
* reversed()
* softmax for n>2；sigmoid for n = 2
* embedding 需先預設n個字x k個維度,經過embedding曾後會把傳進的input轉換，input若有15個值代表有15個字，則embeddin會輸出15x256的向量

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata, string, re, random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
#4/3AHP6U4aofsRnDIA8xLWRBFyt9cXwpvgAoXJ5S0I8VZIrhMxQz-JW-A

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
all_files =  os.walk('/content')
for _ in all_files:
  if '%s-%s.txt' % ('eng', 'fra') in _[2]:
    print('%s/%s' % (_[0],_[2][_[2].index('%s-%s.txt' % ('eng', 'fra'))]))


/content/drive/My Drive/space_Colab/data/data/eng-fra.txt
/content/drive/My Drive/space_Colab/data_lang/data/eng-fra.txt


In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    #word2index "wordA":index 第幾個加入
    #word2count "wordA":count_nums 出現次數
    #index2word index: "Word"

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/My Drive/space_Colab/data_lang/data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

*   Read text file and split into lines, split lines into pairs
*   Normalize text, filter by length and content
*   Make word lists from sentences in pairs

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['tu es fatigue et moi aussi .', 'you are tired and so am i .']


In [ ]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size):#(4543,256)
    super(EncoderRNN, self).__init__()
    self.hidden_size = hidden_size #256

    self.embedding = nn.Embedding(input_size, hidden_size) #參數1:size of dict(有input_size個字),參數2:size of each vector(每個字有hidden_size個特徵/維度)
                    #4543字x256維
    self.gru = nn.GRU(hidden_size, hidden_size)
    # gru: gated recurrent units
    # 輸出output 及n秒以後的 h_n(output為hidden_t的維度變換，若為雙向則output_size = 字數x 常數 x hidden_size*2)
    # 參數:(input_size, hidden_size)
  def forward(self, input, hidden): #hidden(1x1x hidden_size) 1x1x256
    #input:一個字 size(1x1), value(word對應的index)
    #1x input_size to 1x input_size x hidden_size to 1x1x flattened(input_size*hidden_size)
    #embedding後變成1x256維的向量 然後再view=> 1x1x256
    embedded = self.embedding(input).view(1,1,-1) 
    output = embedded #1x1x 256
    output,hidden = self.gru(output, hidden) #輸出output 1x1x256, hidden 1x1x256
    return output, hidden

  def initHidden(self):
    return torch.zeros(1,1,self.hidden_size, device=device) #1x1x hidden_size


In [ ]:
input = torch.randn(1, 3, 3)
# input = input.view(1,1,-1)
# input = input.unsqueeze(0)
input.size()

torch.Size([1, 3, 3])

In [ ]:
rnn = nn.GRU(10, 10)
input = torch.randn(1, 1, 10)
h0 = torch.randn(1, 1, 10)
output, hn = rnn(input, h0)
output.size(),hn.size()

(torch.Size([1, 1, 10]), torch.Size([1, 1, 10]))

In [ ]:
class DecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(DecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(output_size, hidden_size)
    #embedding的參數(num_embeddings, embedding_dim)：
    # size of the dictionary of embeddings， the size of each embedding vector
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input, hidden):
    output = self.embedding(input).view(1,1,-1) #1x1x flattened(output_size*hidden_size)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    output = self.softmax(self.out(output[0]))
    return output, hidden
#output[0]?
  
  def initHidden(self):
    return torch.zeros(1,1,self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
    super(AttnDecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout_p = dropout_p
    self.max_length = max_length

    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    #英文2803個字x256維
    #參數1:size of dict(有output_size個字),參數2:size of each vector(每個字有hidden_size個特徵/維度)
    self.attn = nn.Linear(self.hidden_size*2, self.max_length) #?x?x 512 to ?x?x 10
    self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout_p)
    self.gru = nn.GRU(self.hidden_size, self.hidden_size)
    self.out = nn.Linear(self.hidden_size, self.output_size)

  def forward(self, input, hidden, encoder_outputs): #第一個input：1x1； hidden：1x1xhidden, encoder_outputs：10xhidden_size
    #input:一個字 size(1x1), value(word對應的index)
    #embedding後變成1x256維的向量 然後再view=> 1x1x256
    embedded = self.embedding(input).view(1,1,-1) #1x1x 256 
    embedded = self.dropout(embedded)

    attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1) #embedded[0]代表(1x1x256=>1x256)，故 1x256 cat 1x256 =>1x512 to 1x10
    attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))  #unsqueeze(0)在index=0的維度插入1：[3,3] => [1,3,3]
                # 1x1x10 & 1x10x256 => 1x1x256
    output = torch.cat((embedded[0], attn_applied[0]), 1) #1x256 cat 1x256 = 1x512 
    output = self.attn_combine(output).unsqueeze(0)     #1x512 => 1x256 .unsqueeze(0) => 1x1x256

    output = F.log_softmax(self.out(output[0]), dim=1)   #1x256 => 1x2803 每個字的機率
    # print(output)
    return output, hidden, attn_weights

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
  return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
  indexes = indexesFromSentence(lang, sentence)
  indexes.append(EOS_token)
  return torch.tensor(indexes, dtype = torch.long, device = device).view(-1,1)
  # nx1
def tensorsFromPair(pair):
  input_tensor = tensorFromSentence(input_lang, pair[0])
  target_tensor = tensorFromSentence(output_lang, pair[1])
  return (input_tensor, target_tensor)
  #input_tensor:[[1],[5],...,[1]]


In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion, max_length=MAX_LENGTH):
  encoder_hidden = encoder.initHidden()

  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  input_length = input_tensor.size(0) #size(0)：第一個維度，表示有幾個字
  target_length = target_tensor.size(0)

  encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
  # 10 x 256 (10個字，每個字用256維表示)
  loss = 0

  for ei in range(input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden) #input_tensor[ei]表示只傳一個字的index進去 size(1x1)
    encoder_outputs[ei] = encoder_output[0,0] #[0,0]表示從1x1x256變成1x256

  decoder_input = torch.tensor([[SOS_token]], device=device) #1x1 value = 0

  decoder_hidden = encoder_hidden

  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

  if use_teacher_forcing:
    # Teacher forcing: Feed the target as the next input
    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
      #1x2803(個字)的向量, 1x256, 1x10(每句十個字)
      # print(decoder_output, target_tensor[di])
      loss += criterion(decoder_output, target_tensor[di]) #size(1xn) size(1x1) 會從前面一維長向量裡面找出index=後方值的該值
      decoder_input = target_tensor[di]

  else:
    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
      topv,topi = decoder_output.topk(1)
      decoder_input = topi.squeeze().detach()#?
      # print(decoder_output, target_tensor[di])
      loss += criterion(decoder_output, target_tensor[di])
      if decoder_input.item() == EOS_token:
        break
  loss.backward()

  encoder_optimizer.step()
  decoder_optimizer.step()

  return loss.item()/target_length



In [ ]:
# hidden_size = 256
# learning_rate=0.005
# encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
# decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
# training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(10)]
# criterion = nn.NLLLoss()

# iter=1
# training_pair = training_pairs[iter-1]
# input_tensor = training_pair[0]
# target_tensor = training_pair[1]

# # target_tensor
# loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

In [ ]:
torch.tensor([[SOS_token]], device=device)

tensor([[0]])

In [ ]:
import time, math

def asMinutes(s):
  m=math.floor(s/60)
  s -= m*60
  return '%dm %ds' %(m, s)

def timeSince(since, percent):
  now = time.time()
  s = now - since
  es = s/(percent)
  re = es - s
  return '%s (-%s)' % (asMinutes(s), asMinutes(re))
  

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
  start = time.time()
  plot_losses = []
  print_loss_total = 0 #Reset every print_every
  plot_loss_total = 0 #Reset evry
  
  encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
  training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
  criterion = nn.NLLLoss() #第一個參數必為一維長向量(1xn)，第二個參數為1x1的值(該值對應為前方長向量的index) 

  for iter in range(1, n_iters + 1):
    training_pair = training_pairs[iter-1]
    input_tensor = training_pair[0]
    target_tensor = training_pair[1]

    loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    plot_loss_total += loss

    if iter%print_every == 0:
      print_loss_avg = print_loss_total/print_every
      print_loss_total = 0
      print('%s(%d %d%%) %.4f' %(timeSince(start, iter/n_iters), iter, iter/n_iters*100, print_loss_avg))

    if iter%plot_every ==0:
      plot_loss_avg = plot_loss_total/ plot_every
      plot_losses.append(plot_loss_avg)
      plot_loss_total = 0

  showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
  plt.figure()
  fig, ax = plt.subplots()
  # this locator puts ticks at regular intervals
  loc = ticker.MultipleLocator(base=0.2)
  ax.yaxis.set_major_locator(loc)
  plt.plot(points)
  

In [ ]:
def evaluate(encoder, decoder, sentence, max_length = MAX_LENGTH):
  with torch.no_grad():
    input_tensor = tensorFromSentence(input_lang, sentence)
    input_length = input_tensor.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    for ei in range(input_length):
      encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
      encoder_outpus[ei] += encoder_output[0,0]
    
    decoder_input = torch.tensor([[SOS_token]], device = device)
    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
      decoder_attentions[di] = decoder_attention.data
      topv, topi = decoder_output.data.topl(1)
      if topi.item() == EOS_token:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(output_lang.index2word[topi.item()])

      decoder_input = topi.squeeze().detach()

    return decoded_words, decoder_attentions[:di+1]
    

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
  for i in range(n):
    pair = random.choice(pairs)
    print('>', pair[0])
    print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, pair[0])
    output_sentence = ' ',join(output_words)
    print('<', output_sentence)
    print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)


4m 17s (-60m 11s)(5000 6%) 2.8812
8m 30s (-55m 17s)(10000 13%) 2.4179
12m 41s (-50m 44s)(15000 20%) 2.1703
16m 53s (-46m 27s)(20000 26%) 2.0507
21m 8s (-42m 17s)(25000 33%) 1.9702
25m 23s (-38m 4s)(30000 40%) 1.9080
29m 35s (-33m 48s)(35000 46%) 1.8653
33m 46s (-29m 33s)(40000 53%) 1.8171
37m 58s (-25m 19s)(45000 60%) 1.8232
42m 11s (-21m 5s)(50000 66%) 1.8266
46m 23s (-16m 52s)(55000 73%) 1.8782
50m 34s (-12m 38s)(60000 80%) 1.8561
54m 44s (-8m 25s)(65000 86%) 1.8575
58m 56s (-4m 12s)(70000 93%) 1.8770
63m 7s (-0m 0s)(75000 100%) 1.9273
